In [101]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tcn import TCN, tcn_full_summary

# Copy Memory

In this task, each input sequence has length T + 20. The first 10 values are chosen randomly among the digits 1, . . . , 8, with the rest being all zeros, except for the last 11 entries that are filled with the digit ‘9’ (the first ‘9’ is a delimiter). The goal is to generate an output of the same length that is zero everywhere except the last 10 values after the delimiter, where the model is expected to repeat the 10 values it encountered at the start of the input.

Ex.

Input  = 1234567876 000000000 99999999999 
         
         Message        T     Delimeter+Space

Output = 0000000000 000000000 01234567876

                              Message


In [2]:
def CreateCopyMemoryDataset(T, ENTRIES):
    """
    Inputs:
    T       (INT): The number of 0s between the message and delimeter
    ENTRIES (INT): The number of copy memory example entries to make
    
    Outputs:
    X       (np.array, shape:(Entries, T+20)): Copy memory input sequences
    y       (np.array, shape:(Entries, T+20)): Copy memoty output sequences
    """
    
    #Create random integer messages
    RandIntMessages = np.random.randint(low=1, high=8, size=((ENTRIES, 10)))
    
    #Create delimeter
    Delimeter = np.zeros((ENTRIES, 11)) + 9
    
    #Create spacer
    Spacer = np.zeros((ENTRIES, T-1))
    
    #Create X
    # [Message, Spacer, Delimeter]
    X = np.hstack((RandIntMessages, Spacer, Delimeter))
    
    #Create empty array for y,then transplant the X messages to the correct location
    y = np.zeros((ENTRIES, T+20))
    y [:, -10:] = X[:,:10]
        
    return X, y

In [4]:
X, y = CreateCopyMemoryDataset(3, 10)
print(X.shape)
print(y.shape)

print(X[0])
print(y[0])

(10, 23)
(10, 23)
[7. 6. 3. 7. 4. 2. 7. 6. 5. 2. 0. 0. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9. 9.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 7. 6. 3. 7. 4. 2. 7. 6. 5. 2.]


# TCN

Implmenting TCN using: ["https://github.com/philipperemy/keras-tcn"]



In [64]:
def buildTCNModel(T):
    """
    Inputs:
    T   (INT): The number of spaces in the sequence
    
    Outputs:
    TCNMdoel: Compiled TCN model (tf.keras.Model)
    """
    #Setup TCN architecture
    tcn_layer = TCN(
        input_shape =(T+20,1),
        nb_filters=32,
        kernel_size=4,
        nb_stacks=1,
        dilations=(1, 2, 4, 8, 16, 32),
        padding='causal',
        use_skip_connections=True,
        dropout_rate=0.1,
        return_sequences=True,
        activation='relu',
        kernel_initializer='he_normal',
        use_batch_norm=False,
        use_layer_norm=False,
        use_weight_norm=True
    )

    #Create a Tensorflow model
    TCNModel = tf.keras.models.Sequential([tcn_layer, tf.keras.layers.Dense(1)])
    
    #Setup model
    TCNModel.compile(optimizer="adam", loss="mse")
    
    return TCNModel

# Training and Testing

Proof of concept applying Copy Memory to T=20 

In [179]:
#Define Copy Memory Problem
T = 10

#Create Training Examples
ENTRIES = 10000
X, y = CreateCopyMemoryDataset(T, ENTRIES)
print(X.shape)
print(y.shape)

#Build Model
TCNModel = buildTCNModel(T)

(10000, 30)
(10000, 30)


In [180]:
#Train Model
history = TCNModel.fit(X, y, epochs = 10,
                      batch_size = 128, steps_per_epoch = 16,
                      validation_split=0.2)

Epoch 1/10
16/16 [==============================] - 8s 198ms/step - loss: 3.0763 - val_loss: 1.0222
Epoch 2/10
16/16 [==============================] - 1s 73ms/step - loss: 1.1965 - val_loss: 0.2610
Epoch 3/10
16/16 [==============================] - 1s 73ms/step - loss: 0.7048 - val_loss: 0.0953
Epoch 4/10
16/16 [==============================] - 1s 69ms/step - loss: 0.5495 - val_loss: 0.0748
Epoch 5/10
16/16 [==============================] - 1s 71ms/step - loss: 0.4582 - val_loss: 0.0518
Epoch 6/10
16/16 [==============================] - 1s 72ms/step - loss: 0.3974 - val_loss: 0.0383
Epoch 7/10
16/16 [==============================] - 1s 75ms/step - loss: 0.3676 - val_loss: 0.0360
Epoch 8/10
16/16 [==============================] - 1s 71ms/step - loss: 0.3235 - val_loss: 0.0296
Epoch 9/10
16/16 [==============================] - 1s 73ms/step - loss: 0.3001 - val_loss: 0.0289
Epoch 10/10
16/16 [==============================] - 1s 71ms/step - loss: 0.2749 - val_loss: 0.0301


In [118]:
#Create an example and have the trained model predict the output
X, y = CreateCopyMemoryDataset(T, 1)
pred_y = TCNModel.predict(X)

print("Input Sequnce:")
print(y[:,-10:])
print("Output Sequence")
print(np.round(pred_y).reshape(1,-1)[:,-10:])
print("Unrounded-Output Sequnce:")
print(np.round(pred_y,2).reshape(1,-1)[:,-10:])


Input Sequnce:
[[5. 1. 5. 3. 1. 7. 5. 1. 5. 4.]]
Output Sequence
[[4. 2. 4. 4. 2. 5. 5. 3. 5. 4.]]
Unrounded-Output Sequnce:
[[3.9  2.25 4.42 3.69 1.97 5.29 4.57 2.61 4.79 4.01]]


In [177]:
def TCNAccuracy(model, T):
    """
    Inputs
    model: A TCN Model
    T   (INT): The number of spaces in the sequence
    
    Outputs:
    acc (float, [0, 1]): The model's accuracy defined as incorrect digits over total digits for 1000 attempts
    """
    
    #Create Examples
    X, y = CreateCopyMemoryDataset(T, 1000)
    
    #Predict on Examples
    pred_y = model.predict(X)
    
    #Round predictions
    pred_y = np.round(pred_y)
    
    #Count up all digits
    m, n = X.shape
    digits = m*n
    
    #Count up misses
    misses = 0
    for i in range(m):
        for j in range(n):
            if pred_y[i,j] != y[i,j]:
                misses+=1
            
    
    #Calculate Accuracy
    acc = 1 - (misses / digits) 
    
    return acc

# Receptive Field Analysis

TCN Networks are great at providing a large receptive field to provide insight into future predections and effectively transfering information from far away inputs. Effectively, they have a long memory and can apply it effectively. 

In [193]:
def TCN_T_Test(TList):
    """
    Inputs:
    T: A list of integers defining at what intervals of T to test the TCN architecture
    
    Outputs:
    accDF : A pandas dataframe tracking accuracy over training iterations
    lossDF: A pandas dataframe tracking loss over training iterations
    """
    
    # Repeat example generation, training, and testing at each interval T
    for i, T in enumerate(TList):
        
        #Create Training examples
        
        #Train Model
        
        #Calculate Accuracy
       
    
    
    

T=[5,25,50,100,500]
TCN_T_Test(T)


5
25
50
100
500
